In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2, random, time, shutil, csv
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
np.random.seed(42)
%matplotlib inline 

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img

import gc
import skimage.io

In [2]:
train_dir = '../../data/train_keras/'
train_labels = pd.read_csv('../../data/train_labels.csv')

target, dog_breeds = pd.factorize(train_labels['breed'], sort = True)
train_labels['target'] = target

dog_breeds = sorted(list(set(train_labels['breed'])))
n_classes = len(dog_breeds)

class_to_num = dict(zip(dog_breeds, range(n_classes)))

In [13]:
def images_to_array(data_dir, labels_dataframe, img_size = (224,224,3)):
    images_names = labels_dataframe['id']
    images_labels = labels_dataframe['breed']
    data_size = len(images_names)
    #initailize output arrays.
    X = np.zeros([data_size, img_size[0], img_size[1], img_size[2]], dtype=np.uint8)
    y = np.zeros([data_size,1], dtype=np.uint8)
    #read data and lables.
    for i in tqdm(range(data_size)):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name+'.jpg')
        img_pixels = load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
        image_breed = images_labels[i]
        y[i] = class_to_num[image_breed]
    
    y = to_categorical(y)
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('Ouptut Data Size: ', X.shape)
    print('Ouptut Label Size: ', y.shape)
    return X, y

def get_features(model_name, data_preprocessor, input_size, data):
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    #Extract feature.
    feature_maps = feature_extractor.predict(data, batch_size=128, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_extractor

In [12]:
img_size = (224,224,3)
FOLDER = './'
try:
  X = np.load(FOLDER + "X_save.npy")
  y = np.load(FOLDER + "y_save.npy")
  print(len(X), len(y))
except: 
  #img_size chosen to be 331 to suit the used architectures.
  X, y = images_to_array(train_dir, train_labels, img_size)
  np.save(FOLDER + "X_save", X)
  np.save(FOLDER + "y_save", y)
  
train_X = X[:5000]
train_y = y[:5000]
val_X = X[5000:6000]
val_y = y[5000:6000]

20580 20580


In [14]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input

inceptionV3 = InceptionV3(weights='imagenet', include_top=False, input_shape=img_size)
inception_features = get_features(InceptionV3,
                                  inception_preprocessor,
                                  img_size, train_X)

40/40 [==============================] - 85s 2s/step
Feature maps shape:  (5000, 2048)


In [31]:
from tensorflow.keras.applications import EfficientNetB0
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import layers

img_shape = img_size
def build_model(base_model):
    inputs = layers.Input(shape=img_shape)
    x = base_model(inputs, training=False)

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(2, activation="softmax", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name='eff')

    return model

inception_preprocessor = preprocess_input
base_model = EfficientNetB0(include_top=False, input_shape=img_shape, weights="imagenet")
base_model.trainable = False
model = build_model(base_model, 'eff')
# Compile
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
loss = keras.losses.SparseCategoricalCrossentropy()
metrics = keras.metrics.SparseCategoricalAccuracy()
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=[metrics]
)

model.summary()

Model: "eff"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 avg_pool (GlobalAveragePool  (None, 1280)             0         
 ing2D)                                                          
                                                                 
 batch_normalization_847 (Ba  (None, 1280)             5120      
 tchNormalization)                                               
                                                                 
 top_dropout (Dropout)       (None, 1280)              0         
                                                                 
 pred (Dense)                (None, 2)                 2562    

In [ ]:
model.fit(train_ds, epochs=5, validation_data=validation_ds)

In [34]:
from gradcam import Gradcam

img_path = "../images/example.jpg"

gc = Gradcam(model, 
             layer_name="top_conv",
             img_path=img_path,
             size=img_size,
             inner_model=model.get_layer("efficientnetb0"))

gc.generate_stack_img(save_name="../output/example_out")

TypeError: argument 1 must be sequence of length 2, not 3